In [ ]:
import torch
import cv2
import ipywidgets
from cnn import *
import numpy as np
cnn=CNN()
# 加载网络模型
cnn.load_state_dict(torch.load('weight/pytorchp.pkl',map_location='cpu'))

In [ ]:
# def findRoi(frame, thresValue):
#     rois = []
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray1 = cv2.dilate(gray,None,iterations=2)
#     gray2 = cv2.erode(gray1,None,iterations=2)
#     edges = cv2.absdiff(gray,gray2)
#     x = cv2.Sobel(edges,cv2.CV_16S,1,0)
#     y = cv2.Sobel(edges,cv2.CV_16S,0,1)
#     absX = cv2.convertScaleAbs(x)
#     absY = cv2.convertScaleAbs(y)
#     dst = cv2.addWeighted(absX,0.5,absY,0.5,0)
#     ret, ddst = cv2.threshold(dst,thresValue,255,cv2.THRESH_BINARY)
#     contours, hierarchy = cv2.findContours(ddst,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     area = []
#     rois = []
#     for k in range(len(contours)):
#         area.append(cv2.contourArea(contours[k]))
#     if area  :  
#         max_idx = np.argmax(np.array(area))
#         x, y, w, h = cv2.boundingRect(contours[max_idx])
#         rois.append((x,y,w,h))
#     return rois, edges,area
def findRoi(frame, thresValue,kernel):
    rois = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.dilate(gray,kernel,iterations=2)
    # gray2 = cv2.erode(gray1,None,iterations=2)
    ret, ddst = cv2.threshold(gray1,thresValue,255,cv2.THRESH_BINARY_INV)
    # contours, hierarchy = cv2.findContours(ddst,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(ddst,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = []
    rois = []
    for k in range(len(contours)):
        area.append(cv2.contourArea(contours[k]))
    if area  :  
        max_idx = np.argmax(np.array(area))
        x, y, w, h = cv2.boundingRect(contours[max_idx])
        rois.append((x,y,w,h))
    return rois, ddst,area
    
    
    

def findDigit(cnn, roi, thresValue):
    ret, th = cv2.threshold(roi, thresValue, 255, cv2.THRESH_BINARY)
    th = cv2.resize(th,(28,28),interpolation=cv2.INTER_AREA)
    newing=[[th]]
    newing = torch.Tensor(np.array(newing)).float()/255. 
    predicions=cnn(newing)
    retult=torch.argmax(predicions).detach().numpy()
    return retult,th
def concatenate(images):
    n = len(images)
    output = np.zeros(20*20*n).reshape(-1,20)
    for i in range(n):
        output[20*i:20*(i+1),:] = images[i]
    return output

In [ ]:
font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN#字体类型
kernel = np.ones((4, 4), np.uint8)
wid = ipywidgets.Image()
cap=cv2.VideoCapture(0)
if not cap.isOpened():
    cap=cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError('Can not open video')
cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,160)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,120)    
ret,img = cap.read()    
cntr=0;
display(wid)
while True:
    ret,frame=cap.read()
    cntr=cntr+1;
    #if((cntr%2)==0):
    if(1):
        rois,edges,area = findRoi(frame, 70,kernel)
        digits = []
        if rois: 
            x, y, w, h=rois[0]
            x_small=int(x+(w-h)/2)
            if  x_small<0:
                 x_small = 0
            # digit,th = findDigit(cnn,edges[y:y+h,x:x+w], 70)
            digit,th = findDigit(cnn,edges[y:y+h, x_small: x_small+h], 20)
            cv2.rectangle(frame, (int(x+(w-h)/2),y), (int(x+(w+h)/2),y+h), (153,153,0), 2)
            cv2.putText(frame, str(digit), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2)
        #cv2.putText(frame, str(digit), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2)
        newEdges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        newFrame = np.hstack((frame,newEdges))
        wid.value = cv2.imencode('.jpg',newFrame)[1].tobytes()

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

class Pid():
    def __init__(self,  kp, ki, kd):
        self.KP = kp
        self.KI = ki
        self.KD = kd
        self.now_err = 0
        self.last_err = 0
        self.last_last_err = 0
        self.change_val = 0
        self.out_val=0
    def cmd_pid(self,exp_val,now_val):
        self.last_last_err = self.last_err
        self.last_err = self.now_err
        self.now_err = exp_val - now_val
        self.change_val = self.KP * (self.now_err - self.last_err) + self.KI * \
            self.now_err + self.KD * (self.now_err - 2 * self.last_err
                                      + self.last_last_err)
        self.out_val += self.change_val
        return self.out_val


pid_val_x = []
pid_val_y = []
my_Pid = Pid( 0.1, 0.0, 0.0)
tarx=5
tary=5
def findRoi(frame, thresValue,kernel):
    rois = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, ddst = cv2.threshold(gray,thresValue,255,cv2.THRESH_BINARY_INV)
    gray1 = cv2.erode(ddst,None,iterations=2)
    gray2 = cv2.dilate(gray1,kernel,iterations=2)

   
    # contours, hierarchy = cv2.findContours(ddst,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(gray2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = []
    rois = []
    for k in range(len(contours)):
        area.append(cv2.contourArea(contours[k]))
    if area  :  
        max_idx = np.argmax(np.array(area))
        x, y, w, h = cv2.boundingRect(contours[max_idx])
        rois.append((x,y,w,h))
    return rois, gray2
def distance_to_camera(knownDistance, focalLength, perWidth):
    # compute and return the distance from the maker to the camera
    return  (perWidth * knownDistance)/ focalLength    
# initialize the known distance from the camera to the object, which
# in this case is 24 inches
KNOWN_DISTANCE = 10
 
# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 4
 
# load the furst image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize
# the focal length 
thresValue=60
kernel=np.ones((10,10),dtype=np.uint8)
cap=cv2.VideoCapture(0)
frame_height = cap.get(4)
frame_width = cap.get(3)
while cap.isOpened(): 
    ret, frame=cap.read()    
    rois, ddst=findRoi(frame,thresValue,kernel)
    if rois: 
        x, y, w, h=rois[0]
        x_small=int(x+(w-h)/2)
        if  x_small<0:
                x_small = 0
        # digit,th = findDigit(cnn,edges[y:y+h,x:x+w], 70)
        cv2.rectangle(frame, (x,y), (int(x+w),y+h), (153,153,0), 2)
        cv2.putText(frame, "%.2fft" % width,
        (frame.shape[1] - 200, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
        2.0, (0, 255, 0), 3)
    width=distance_to_camera(KNOWN_DISTANCE,500,x-(frame.shape[1]/2))   
    newEdges = cv2.cvtColor(ddst, cv2.COLOR_GRAY2BGR)
    newFrame = np.hstack((frame,newEdges))  
    cv2.imshow('newFrame',newFrame)    
    print("\r{},{},{},{},{}".format(x, y, w, h,width),end='')
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()


0,0,640,480,-6.4.2222

In [ ]:
import matplotlib.pyplot as plt

from IPython import display

class Pid():
    def __init__(self,  kp, ki, kd):
        self.KP = kp
        self.KI = ki
        self.KD = kd
        self.now_err = 0
        self.last_err = 0
        self.last_last_err = 0
        self.change_val = 0
        self.out_val=0
    def cmd_pid(self,exp_val,now_val):
        self.last_last_err = self.last_err
        self.last_err = self.now_err
        self.now_err = exp_val - now_val
        self.change_val = self.KP * (self.now_err - self.last_err) + self.KI * \
            self.now_err + self.KD * (self.now_err - 2 * self.last_err
                                      + self.last_last_err)
        self.out_val += self.change_val
        return self.out_val


pid_val = []
my_Pid = Pid( 0.1, 0.15, 0.1)
for i in range(0, 300):
    pid_val.append(my_Pid.cmd_pid(1000000,))
plt.plot(pid_val)
plt.show()


In [ ]:
'''
动态折线图演示示例
'''
 
import numpy as np
import matplotlib.pyplot as plt
 
plt.ion()
plt.figure(1)
t_list = []
result_list = []
t = 0
 
while True:
 if t >= 10 * np.pi:
  plt.clf()
  t = 0
  t_list.clear()
  result_list.clear()
 else:
  t += np.pi / 4
  t_list.append(t)
  result_list.append(np.sin(t))
  plt.plot(t_list, result_list,c='r',ls='-', marker='o', mec='b',mfc='w') ## 保存历史数据
  #plt.plot(t, np.sin(t), 'o')
  plt.pause(0.1)

In [ ]:
# import torch
import matplotlib.pyplot as plt
from IPython import display
x_list = []
y_list = []
for i in range(100):
    x_list.append(i)
    y_list.append(i**2)
    display.clear_output(wait=True)
    plt.plot(x_list, y_list)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    plt.pause(0.1)


In [ ]:
import torch
flag = torch.cuda.is_available()
print(flag)
 
ngpu= 1
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
print(torch.cuda.get_device_name(0))
print(torch.rand(3,3).cuda()) 

In [9]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Image
img = 'https://ultralytics.com/images/zidane.jpg'

# Inference
results = model(img)


Using cache found in C:\Users\lenovo/.cache\torch\hub\ultralytics_yolov5_master
requirements: numpy>=1.18.5 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

requirements: 1 package updated per C:\Users\lenovo\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2022-6-14 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


SSLError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /ultralytics/yolov5/master/data/images/zidane.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))